# VISMAY SHAH JUPYTER NOTEBOOK FOR INSIDE PETROLEUM TEST 2

In [147]:
import pandas as pd
import numpy as np
import math

In [148]:
desi = 0.8412              #Initial Effective Secant Decline per year
b = 0.63
wi = 0.75
rev = 0.60
oil_price = 48.5           #USD/bbl
discount_factor = 0.10
capex = 5500000            #USD
opex = 60000               #USD/year

start_day = 0              #day
qi = 2759.15               #bbl/day
q_end = 0.8                #bbl/day
d_switch = 0.08            #per year
end_day = 16441            #day

In [149]:
Di = round((((1-desi)**(-b))-1)/b, 3)   #Initial Nominal Decline per year
Di

3.472

# Find t_switch and q_switch

In [153]:
eco_df = pd.DataFrame(columns=["time", "q", "decline_rate", "cum_prod", "ann_prod", 
                               "capex", "opex", "revenue", "disc_fac","cash_flow"])
eco_df

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow


In [154]:
eco_df["time"] = np.arange(0, end_day)
eco_df.head()

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [159]:
eco_df.iloc[0,1:] = [qi,round(Di/365,5),0,0,(capex*wi),0,0,1,-(capex*wi)]

In [160]:
eco_df.head(3)

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
0,0,2759.15,0.00951,0,0,4.125e+06,0,0,1,-4.125e+06
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [161]:
for i in range(0,end_day-1):
    eco_df.iloc[i+1, 2] = round(((Di/365)/(1 + ((Di/365) * b * (i+1)))), 5)
    
    if eco_df.iloc[i+1, 2] < round(d_switch/365, 5):
        eco_df.iloc[i+1, 2] = round(d_switch/365, 5)

In [162]:
eco_df.iloc[[6887,6888]]

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
6887,6887,NaN,0.00023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6888,6888,NaN,0.00022,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We can see that on tha day 6888 the production decline curve changed from hyperbolic to exponential. So after that the value of b = 0 and Di will remain constant at 0.08

In [163]:
# updating the values of decline rate until day 6888 again
for i in range(0,len(eco_df[eco_df["decline_rate"] > round(d_switch/365, 5)])-1):
    eco_df.iloc[i+1, 1] = round(qi*((1 + ((Di/365) * b * (i+1)))**(-1/b)), 2)
    
# updating the values of decline rate after the day 6888 to be constant at 0.00022 per day
for i in range(len(eco_df[eco_df["decline_rate"] > round(d_switch/365, 5)]), end_day-1):
    eco_df.iloc[i+1, 1] = round(7.24 * math.exp(-round(d_switch/365, 5) * (eco_df.iloc[i+1, 0] - 6887)), 2)

In [164]:
eco_df.tail()

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
16436,16436,0.89,0.00022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16437,16437,0.89,0.00022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16438,16438,0.89,0.00022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16439,16439,0.89,0.00022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16440,16440,0.89,0.00022,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Find NPV and ROI

To find the cum_prod and ann_prod it is necessary to convert the data into years from days. t_switch is actually on day 6888 but if we see that there is not a huge difference between flow rates on day 6888 and 6935, we will use 6935 as t_switch that is year number 19. I have alos ignored last 16 days of production from 16425 to 16441 as they do not add significant value towards the revenue or NPV and to ease the coding as well 

In [165]:
eco_df.iloc[6935]

time               6935
q                  7.16
decline_rate    0.00022
cum_prod            NaN
ann_prod            NaN
capex               NaN
opex                NaN
revenue             NaN
disc_fac            NaN
cash_flow           NaN
Name: 6935, dtype: object

In [166]:
year_df = pd.DataFrame(columns=eco_df.columns)
year_df

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow


In [167]:
year_df["time"] = np.arange(0, 16441, 365)
year_df.head()

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1095,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [168]:
year_df.iloc[0,:] = [0,qi,round(Di,2),0,0,round(capex*wi,2),0,0,1,round(-capex*wi,2)]

In [170]:
year_df.head(3)

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
0,0,2759.15,3.47,0,0,4.125e+06,0,0,1,-4.125e+06
1,365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,730,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [171]:
year_df["capex"].fillna(value = 0, inplace = True) #Filling all Capex values with "0"
year_df["opex"].fillna(value = opex*wi, inplace = True) #Filling up all the Opex values considering working interest
year_df.head(3)

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
0,0,2759.15,3.47,0,0,4125000.0,0.0,0,1,-4.125e+06
1,365,NaN,NaN,NaN,NaN,0.0,45000.0,NaN,NaN,NaN
2,730,NaN,NaN,NaN,NaN,0.0,45000.0,NaN,NaN,NaN


In [172]:
# Production rate, decline rate, and cummulative production till year 19 using hyperbolic decline equation 
for i in range(0,20):
    year_df.iloc[i+1, 1] = round(qi*((1 + (Di * b * (i+1)))**(-1/b)), 2)                                          #q
    year_df.iloc[i+1, 2] = round((Di/(1 + (Di * b * (i+1)))), 2)                                                  #decline_rate
    year_df.iloc[i+1, 3] = round(((1/(b*Di*((1/b)-1)))*(qi-year_df.iloc[i+1,1]*(1 + (Di * b * (i+1)))))*365, 2)   #cum_prod


# Production rate, decline rate, and cummulative production after year 19 to the end using exponential decline equation 
for i in range(19, len(year_df)-1):
    year_df.iloc[i+1, 1] = round(year_df.iloc[19,1] * math.exp(-round(d_switch, 2) * (i+1 - 19)), 2)              #q
    year_df.iloc[i+1, 2] = round(d_switch, 2)                                                                     #decline_rate
    year_df.iloc[i+1, 3] = round((((7.16-year_df.iloc[i+1,1])/d_switch)*365)+year_df.iloc[19,3], 2)               #cum_prod
    



for i in range(0,len(year_df)-1):
    year_df.iloc[i+1, 4] = round(year_df.iloc[i+1, 3] - year_df.iloc[i, 3], 2)                    #Annual Production
    year_df.iloc[i+1, 7] = round(year_df.iloc[i+1, 4] * oil_price * rev, 2)                       #Revenue
    year_df.iloc[i+1, 8] = round(year_df.iloc[i, 8]+(year_df.iloc[i, 8]*discount_factor), 2)      #Discount Factor
    year_df.iloc[i+1, 9] = round((year_df.iloc[i+1, 7]-year_df.iloc[i+1, 6]-year_df.iloc[i+1, 5])/year_df.iloc[i+1, 8], 2) 
                                                                                                                #Cash_Flow

    
year_df

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow
0,0,2759.15,3.47,0,0,4125000.0,0.0,0,1,-4.125e+06
1,365,438.21,1.09,387098,387098,0.0,45000.0,1.12646e+07,1.1,1.01996e+07
2,730,191.2,0.65,491966,104867,0.0,45000.0,3.05164e+06,1.21,2.48482e+06
3,1095,111.2,0.46,545024,53058.6,0.0,45000.0,1.54401e+06,1.33,1.12707e+06
4,1460,74.3,0.36,578131,33106.4,0.0,45000.0,963395,1.46,629038
5,1825,53.88,0.29,601210,23079.2,0.0,45000.0,671604,1.61,389195
6,2190,41.25,0.25,618409,17199.1,0.0,45000.0,500493,1.77,257340
7,2555,32.82,0.21,631842,13432.8,0.0,45000.0,390894,1.95,177382
8,2920,26.88,0.19,642665,10823.6,0.0,45000.0,314966,2.15,125565
9,3285,22.51,0.17,651644,8979.14,0.0,45000.0,261293,2.36,91649.6


In [173]:
print("NPV of the project = {} USD".format(year_df["cash_flow"].sum()))
print("ROI of the project = {}".format(((((year_df["cash_flow"].sum())/(capex*wi))+1)*100)))

NPV of the project = 11615925.85 USD
ROI of the project = 381.5982024242424


# New Oil Prices, Cash Flow, NPV, and ROI

In [174]:
year_df = year_df.assign(new_oil_prices = np.nan, 
                         new_revenue    = np.nan, 
                         new_cash_flow  = np.nan)

In [175]:
year_df.head(3)

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow,new_oil_prices,new_revenue,new_cash_flow
0,0,2759.15,3.47,0,0,4125000.0,0.0,0,1,-4.125e+06,NaN,NaN,NaN
1,365,438.21,1.09,387098,387098,0.0,45000.0,1.12646e+07,1.1,1.01996e+07,NaN,NaN,NaN
2,730,191.2,0.65,491966,104867,0.0,45000.0,3.05164e+06,1.21,2.48482e+06,NaN,NaN,NaN


In [179]:
year_df.loc[0,["new_oil_prices","new_revenue","new_cash_flow"]] = [0, 0, -capex*wi]
year_df.loc[1,"new_oil_prices"] = oil_price

In [182]:
year_df.head(3)

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow,new_oil_prices,new_revenue,new_cash_flow
0,0,2759.15,3.47,0,0,4125000.0,0.0,0,1,-4.125e+06,0.0,0.0,-4125000.0
1,365,438.21,1.09,387098,387098,0.0,45000.0,1.12646e+07,1.1,1.01996e+07,48.5,NaN,NaN
2,730,191.2,0.65,491966,104867,0.0,45000.0,3.05164e+06,1.21,2.48482e+06,NaN,NaN,NaN


In [185]:
for i in range(1,len(year_df)-1):
    year_df.iloc[i+1, 10] = round(year_df.iloc[i, 10] - 1, 2)                                                    #new_oil_prices
    
for i in range(0,len(year_df)-1):
    year_df.iloc[i+1, 11] = round(year_df.iloc[i+1, 4] * year_df.iloc[i+1,10] * rev, 2)                            #new_revenue
    year_df.iloc[i+1, 12] = round((year_df.iloc[i+1, 11]-year_df.iloc[i+1, 6]-year_df.iloc[i+1, 5])/year_df.iloc[i+1, 8], 2) 
                                                                                                                   #new_cashflow
    

year_df

,time,q,decline_rate,cum_prod,ann_prod,capex,opex,revenue,disc_fac,cash_flow,new_oil_prices,new_revenue,new_cash_flow
0,0,2759.15,3.47,0,0,4125000.0,0.0,0,1,-4.125e+06,0.0,0.00,-4125000.00
1,365,438.21,1.09,387098,387098,0.0,45000.0,1.12646e+07,1.1,1.01996e+07,48.5,11264565.77,10199605.25
2,730,191.2,0.65,491966,104867,0.0,45000.0,3.05164e+06,1.21,2.48482e+06,47.5,2988715.20,2432822.48
3,1095,111.2,0.46,545024,53058.6,0.0,45000.0,1.54401e+06,1.33,1.12707e+06,46.5,1480336.06,1079200.05
4,1460,74.3,0.36,578131,33106.4,0.0,45000.0,963395,1.46,629038,45.5,903803.90,588221.85
5,1825,53.88,0.29,601210,23079.2,0.0,45000.0,671604,1.61,389195,44.5,616214.37,354791.53
6,2190,41.25,0.25,618409,17199.1,0.0,45000.0,500493,1.77,257340,43.5,448895.47,228189.53
7,2555,32.82,0.21,631842,13432.8,0.0,45000.0,390894,1.95,177382,42.5,342536.14,152582.64
8,2920,26.88,0.19,642665,10823.6,0.0,45000.0,314966,2.15,125565,41.5,269506.64,104421.69
9,3285,22.51,0.17,651644,8979.14,0.0,45000.0,261293,2.36,91649.6,40.5,218193.10,73386.91


In [187]:
print("NPV of the project = {} USD".format(year_df["new_cash_flow"].sum()))
print("ROI of the project = {}".format(((((year_df["new_cash_flow"].sum())/(capex*wi))+1)*100)))

NPV of the project = 11222441.55 USD
ROI of the project = 372.0591890909091
